In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import math


In [ ]:
def display_unique_vals(df, col_name):
  values = set(df[col_name])
  print(f'values of {col_name}: {values}')

In [ ]:
file_path = '../data/raw/invoices.csv'

dtypes = {'invoice_number' : 'str',
  'invoice_date' : 'str',
  'customer_name' : 'str',
  'item_name' : 'str',
  'quantity' : 'int',
  'price' : 'float',
  'tax_rate' : 'float',
  'total_amount' : 'float',
  }

parse_dates = ['invoice_date']


In [ ]:
df = pd.read_csv(file_path, parse_dates=parse_dates, dtype=dtypes)
display(df)

# Summary statistics

In [ ]:
summary_stats = df.describe(include = 'all')
summary_stats.loc['dtype'] = df.dtypes
summary_stats.loc['size'] = len(df)
summary_stats.loc['nan_vals'] = df.isnull().sum()
display(summary_stats)

display_unique_vals(df, 'invoice_number')
display_unique_vals(df, 'customer_name')
display_unique_vals(df, 'item_name')

# invoice_date

In [ ]:
for key, value in df.groupby(['invoice_number']):
  tmp = value[['invoice_number', 'invoice_date']]
  display(tmp)

Same invoice number seems to mean that there were two transactions same date. Using this we can replace nan values in invoice_date column.

In [ ]:
for key, value in df.groupby(['invoice_number'], dropna=True):
  df_most_occ_val = value[['invoice_date']].mode()
  most_occ_val = df_most_occ_val['invoice_date'].item()
  df.loc[df['invoice_number'] == key, 'invoice_date'] = most_occ_val


# customer_name

In [ ]:
for key, value in df.groupby(['tax_rate']):
  tmp = value[['invoice_number', 'customer_name', 'tax_rate']]
  display(tmp)

It seems that each customer has fixed tax rate. (**Not entirely sure** how the tax rate is defined, so assuming that each company has a specific tax rate and no company share a tax rate it's okay to assign the customer name based on the tax rate).

In [ ]:
for key, value in df.groupby(['item_name']):
  tmp = value[['invoice_number', 'customer_name',	'item_name', 'tax_rate']]
  display(tmp)

Can't rely on item name to fill out customer name as different customers buy same items

In [ ]:
for key, value in df.groupby(['tax_rate'], dropna=True):
  df_most_occ_val = value[['customer_name']].mode()
  most_occ_val = df_most_occ_val['customer_name'].item()
  df.loc[df['tax_rate'] == key, 'customer_name'] = most_occ_val


# price

Based on summary statistics some prices are negative. That shouldn't be possible, unless invoices also include returns, but it's not defined.

In [ ]:
df['price'] = df['price'].abs()

# item_name

In [ ]:
for key, value in df.groupby(['price']):
  tmp = value[['item_name', 'price']]
  display(tmp)

Assuming that no items shares the same price, then we can assign new name to unknown item

In [ ]:
df.loc[df['price'] == 20, 'item_name'] = "Widget E"

# quantity

Based on summary statistics some quantities are negative. That shouldn't be possible, unless invoices also include returns, but it's not defined.

In [ ]:
df['quantity'] = df['quantity'].abs()

# tax_rate

In [ ]:
for key, value in df.groupby(['customer_name']):
  tmp = value[['customer_name', 'tax_rate']]
  display(tmp)

It seems that each customer has fixed tax rate. (**Not entirely sure** how the tax rate is defined, so assuming that each company has a specific tax rate and no company share a tax rate it's okay to assign the customer name based on the tax rate). **bold text**

In [ ]:
for key, value in df.groupby(['customer_name'], dropna=True):
  df_most_occ_val = value[['tax_rate']].mode()
  most_occ_val = df_most_occ_val['tax_rate'].item()
  df.loc[df['customer_name'] == key, 'tax_rate'] = most_occ_val


# total_amount

In [ ]:
filtered_df = df[df['total_amount'].notnull()]
price_before_taxes = filtered_df['quantity'] * filtered_df['price']
price_after_taxes = price_before_taxes + price_before_taxes * filtered_df['tax_rate']
np.isclose(filtered_df['total_amount'], price_after_taxes)

It seems that non nan total amounts are calculated correctly so we can only calculate nan total amount values

In [ ]:
filtered_df = df[df['total_amount'].isnull()]
price_before_taxes = filtered_df['quantity'] * filtered_df['price']
price_after_taxes = price_before_taxes + price_before_taxes * filtered_df['tax_rate']
df.loc[df['total_amount'].isnull(), 'total_amount'] = price_after_taxes

# End result (Summary statistics)

In [ ]:
display(df)

In [ ]:
summary_stats = df.describe(include = 'all')
summary_stats.loc['dtype'] = df.dtypes
summary_stats.loc['size'] = len(df)
summary_stats.loc['nan_vals'] = df.isnull().sum()
display(summary_stats)

display_unique_vals(df, 'invoice_number')
display_unique_vals(df, 'customer_name')
display_unique_vals(df, 'item_name')
display(df)